# Importing important libraries

In [2]:
import time
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re

In [133]:
#connect to times of india webpage
sURL="https://timesofindia.indiatimes.com/"
driver=webdriver.Chrome("/Users/bhupendrakumar/Documents/Personal/chromedriver")
driver.get(sURL)

In [47]:
# Navigate to archives page
link=driver.find_element(By.LINK_TEXT,"Archives")
Archives_Address=link.get_attribute('href')
driver.get(Archives_Address)
time.sleep(3)

In [52]:
# Grab data for Apr 2017
links=driver.find_elements(By.LINK_TEXT,"August")

from datetime import date
d=date.today()
ActiveYear=int(d.strftime("%Y"))
YearToBesearched=2019
Link_Index=ActiveYear-YearToBesearched
driver.get(links[Link_Index-1].get_attribute('href'))

In [54]:
#Store links of days
Days_links=[]
for iDay in range(1,32):
    webaddress=driver.find_element(By.LINK_TEXT,str(iDay)).get_attribute('href')
    Days_links.append(webaddress)

In [55]:
#Store news links
News_links=[]
for iDay in Days_links:
    driver.get(iDay)
    time.sleep(2)
    #Set of news links
    News_Addresses=driver.find_elements_by_xpath("//span[@style='font-family:arial ;font-size:12;color: #006699']")
    
    #News links
    Text_Links=News_Addresses[0].text
    texts=Text_Links.split("\n")
    pattern=r"[a-zA-Z]+"
    for sText in texts:    
        if re.search(pattern,sText):
            webaddress=driver.find_element(By.LINK_TEXT,sText).get_attribute('href')
            News_links.append(webaddress)

In [56]:
len(News_links)

7718

## Defining and intializing lists

In [123]:
Headlines_list=[]
Authors_list=[]
Dates_list=[]
News_list=[]
Verticals_list=[]

In [134]:
#Prev i=5531
i=7139
for sNews_Link in News_links[7139:]:
    i+=1
    print(i)
    driver.get(sNews_Link)
    time.sleep(3)
    try:
        # If error class is present and is empty then add the texts of desired classes.
        driver.find_element_by_xpath("//div[@class='error']")
        if driver.find_element_by_xpath("//div[@class='error']").text =="":
            #Extracting Headlines
            try:
                headline=driver.find_element_by_xpath("//h1[@class='_23498']")
                headline=headline.text
            except ElementNotInteractableException:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                try:
                    headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                    headline=headline.text
                except NoSuchElementException:
                    headline="-"
            Headlines_list.append(headline)

            #Extracting Authors and Dates
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
                if "|" in Author_Date:
                    Authors_list.append(Author_Date[:Author_Date.index(" | ")])
                else:
                    Authors_list.append("-")

                if "Aug" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Aug"):])

            except ElementNotInteractableException:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Aug")])
                if "Aug" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Aug"):])
            except NoSuchElementException:
                try:
                    Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                    Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                    
                    if "Aug" not in Author_Date:
                        Dates_list.append("-")
                        Authors_list.append("-")
                    else:
                        Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Aug")])
                        Dates_list.append(Author_Date[Author_Date.index("Aug"):])
                except NoSuchElementException:
                    Dates_list.append("-")
                    Authors_list.append("-")
            #Extracting Description
            try:
                News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
            except ElementNotInteractableException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            except NoSuchElementException:
                try:
                    News=driver.find_element_by_xpath("//div[@class='Normal']").text
                    News=News.replace("\n","")
                except NoSuchElementException:
                    try:
                        News=driver.find_element_by_xpath("//span[@class='readmore_span']").text
                        News=News.replace("\n","")
                    except NoSuchElementException:
                        News="-"
            News_list.append(News)

            #Extracting Verticals
            try:
                Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
                if len(Vertical)>=2:
                    Vert=Vertical[1].text
                else:
                    Vert="-"
            except ElementNotInteractableException:
                Vert="-"
            except NoSuchElementException:
                Vert="-"

            Verticals_list.append(Vert)
        else: # If error class exists and text data is peresent then probably news has been removed.
              # Continue from the next page
            continue
    except NoSuchElementException: # If no error class exists, then should have no problem in collecting data
        #Extracting Headlines
            try:
                headline=driver.find_element_by_xpath("//h1[@class='_23498']")
                headline=headline.text
            except ElementNotInteractableException:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                try:
                    headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                    headline=headline.text
                except NoSuchElementException:
                    headline="-"
            Headlines_list.append(headline)

            #Extracting Authors and Dates
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
                if "|" in Author_Date:
                    Authors_list.append(Author_Date[:Author_Date.index(" | ")])
                else:
                    Authors_list.append("-")

                if "Aug" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Aug"):])

            except ElementNotInteractableException:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Aug")])
                if "Aug" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Aug"):])
            except NoSuchElementException:
                try:
                    Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                    Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                    
                    if "Aug" not in Author_Date:
                        Authors_list.append("-")
                        Dates_list.append("-")
                    else:
                        Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Aug")])
                        Dates_list.append(Author_Date[Author_Date.index("Aug"):])
                except NoSuchElementException:
                    Dates_list.append("-")
                    Authors_list.append("-")
            #Extracting Description
            try:
                News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
            except ElementNotInteractableException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            except NoSuchElementException:
                try:
                    News=driver.find_element_by_xpath("//div[@class='Normal']").text
                    News=News.replace("\n","")
                except NoSuchElementException:
                    try:
                        News=driver.find_element_by_xpath("//span[@class='readmore_span']").text
                        News=News.replace("\n","")
                    except NoSuchElementException:
                        News="-"
            News_list.append(News)

            #Extracting Verticals
            try:
                Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
                if len(Vertical)>=2:
                    Vert=Vertical[1].text
                else:
                    Vert="-"
            except ElementNotInteractableException:
                Vert="-"
            except NoSuchElementException:
                Vert="-"

            Verticals_list.append(Vert)

7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285
7286
7287
7288
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302
7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339


In [135]:
print(len(News_list))
print(len(Headlines_list))
print(len(Authors_list))
print(len(Dates_list))
print(len(Verticals_list))

2185
2185
2185
2185
2185


In [132]:
Dates_list[len(Dates_list)-1]

'Aug 29, 2019, 14:25 IST'

In [111]:
Headlines_list.pop()
Authors_list.pop()
Dates_list.pop()
News_list.pop()
Verticals_list.pop()

'-'

## Storing Data in CSV file

In [136]:
#Declaring the dataframe
Scrapped_data=pd.DataFrame()

#Initializing columns in the new data frame
Scrapped_data['Date']=Dates_list
Scrapped_data['Author']=Authors_list
Scrapped_data['Vertical']=Verticals_list
Scrapped_data['Headline']=Headlines_list
Scrapped_data['Description']=News_list

In [137]:
#Storing data in csv file, in m ultiple iterations
Scrapped_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Aug2019_data5.csv")

In [138]:
# Merge different data files collected during extraction process
Data_1=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Apr2017_data1.csv")
Data_2=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Apr2017_data2.csv")
Data_3=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Apr2017_data3.csv")
Data_4=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Apr2017_data4.csv")
Data_5=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Apr2017_data5.csv")

In [139]:
Final_data=pd.concat([Data_1,Data_2,Data_3,Data_4,Data_5],ignore_index=True)

In [140]:
# It has 7712 rows and 6 columns
Final_data.shape

(7437, 6)

In [141]:
Final_data.columns

Index(['Unnamed: 0', 'Date', 'Author', 'Vertical', 'Headline', 'Description'], dtype='object')

In [142]:
# Drop unnamed column
Final_data.drop('Unnamed: 0',inplace=True,axis=1)

In [143]:
# Observing first 20 rows of the collected data
Final_data.head(20)

Date               Author       Vertical  \
0   Apr 11, 2017, 11:26 IST        Anup Satphale      CITY NEWS   
1    Apr 1, 2017, 00:00 IST    TNNStuti Agarwal   Entertainment   
2    Apr 1, 2017, 11:10 IST         Asha Prakash      CITY NEWS   
3    Apr 1, 2017, 00:00 IST     Srishti Dasgupta      CITY NEWS   
4    Apr 1, 2017, 00:00 IST        Prathibha Joy      CITY NEWS   
5    Apr 1, 2017, 00:00 IST       Anjana George   Entertainment   
6    Apr 3, 2017, 11:53 IST          P Sangeetha      CITY NEWS   
7    Apr 1, 2017, 00:00 IST   Vinita Chaturvedi       Lifestyle   
8                         -                    -      Lifestyle   
9                         -                    -      CITY NEWS   
10   Apr 1, 2017, 14:19 IST        Sumita Sarkar      CITY NEWS   
11   Apr 1, 2017, 14:15 IST     Santosh Sonawane      CITY NEWS   
12   Apr 1, 2017, 14:10 IST     Santosh Sonawane      CITY NEWS   
13   Apr 1, 2017, 00:04 IST  Chaitanya Deshpande      CITY NEWS   
14   Apr 1, 2017, 14:02 IST                  TNN      CITY NEWS   
15   Apr 2, 2017, 08:21 IST                  TNN      CITY NEWS   
16   Apr 2, 2017, 08:22 IST                  TNN      CITY NEWS   
17   Apr 1, 2017, 14:00 IST  Chaitanya Deshpande      CITY NEWS   
18   Apr 1, 2017, 10:41 IST                  TNN      CITY NEWS   
19   Apr 1, 2017, 10:42 IST          Ardhra Nair      CITY NEWS   

                                             Headline  \
0                 We can't get bogged down by attacks   
1                Here’s how GST will affect your life   
2   Now that the Biennale is over, what happens to...   
3             10 burning hot dishes to try right now!   
4   #BeSafe: Take a selfie and win a prize from ci...   
5     Unni goes on a crash diet to play a 63-year-old   
6                  Coimbatoreans off to cooler climes   
7   Australians started the fire and we returned i...   
8                                                   -   
9                                                   -   
10  Education deptartment tips for junior colleges...   
11             Autorickshaws flout fare-by-meter norm   
12                    Police team to protect tourists   
13              Stage set for Ram Navami celebrations   
14                 Swine flu death toll 16 till March   
15                      Survey to locate liquor shops   
16             Textile units to be solar powered soon   
17  Students transliterate Buddhist text into Devn...   
18  Ten professors, HoDs file application for VC's...   
19  Relief from heatwave condition in western Indi...   

                                          Description  
0   Say foreign students, concerned about hate cri...  
1   What is GST?Most nations, when they start out ...  
2   Whoever has visited the Kochi-Muziris Biennale...  
3   They say that the chilli, like the potato and ...  
4   Who would have thought that a selfie could win...  
5   Actor Unni Mukundan is known for undergoing dr...  
6   As the city braces up for a hot spell of summe...  
7   After the recent Test series win over the Auss...  
8   We have heard of fear of height and fear of wa...  
9   NASHIK: Deputy director of education Ramchandr...  
10  NASHIK: The police department and the Regional...  
11  NASHIK: Tourists visiting the city are set to ...  
12  Nashik: The city is all set to celebrate Ram N...  
13  NASHIK: Swine flu has claimed 16 lives from ac...  
14  KOLHAPUR: The district unit of state excise de...  
15  KOLHAPUR: The state plans to design a scheme t...  
16  NASHIK: The National Mission for Manuscripts (...  
17  PUNE: As many as 10 professors from Savitribai...  
18  PUNE: Respite from the onslaught of scorcher i...  
19  PUNE: Joining the bandwagon of urban reforms i...

In [144]:
#Store Final data in csv format
Final_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/Aug2019_data(TOI).csv")

In [145]:
driver.quit()